In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
import numpy as np

In [2]:
train = pd.read_csv('./data/dacon_g/train.csv', index_col=0)
test = pd.read_csv('./data/dacon_g/test.csv', index_col=0)
sample_submission = pd.read_csv('./data/dacon_g/sample_submission.csv', index_col=0)

In [5]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type_num
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340,8
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314,8
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928,8
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855,8
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655,10


In [4]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [6]:
train_x = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_x = test

In [30]:
train_x1 = train.drop(columns=['fiberID'], axis=1)

In [9]:
%%time
## RF Model
forest = RandomForestClassifier(n_estimators=100, n_jobs=-1,verbose=1)
forest.fit(train_x, train_y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.3min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=1,
                       warm_start=False)

In [10]:
y_pred = forest.predict_proba(test_x)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


In [31]:
d_train = lgbm.Dataset(train_x, label=train_y)
d_train
d1_train = lgbm.Dataset(train_x1, label=train_y)

In [20]:
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':19,
    'metric': 'multi_logloss',
          'is_training_metric': True,
    'learning_rate': 0.01,
          'metric_freq': 3,
    'num_leaves': 35,
         'max_depth':17,
'feature_fraction': 0.8,
'bagging_fraction': 0.7,
'bagging_freq': 14,
'verbose': 1}

In [32]:
%%time
lgb_cv = lgbm.cv(params, d_train, num_boost_round=500, nfold=3, shuffle=True, stratified=True, verbose_eval=100, early_stopping_rounds=30)

nround = lgb_cv['multi_logloss-mean'].index(np.min(lgb_cv['multi_logloss-mean']))
print(nround)

model = lgbm.train(params, d1_train, num_boost_round=nround)

[100]	cv_agg's multi_logloss: 0.951642 + 0.00310641
[200]	cv_agg's multi_logloss: 0.626394 + 0.00375563
[300]	cv_agg's multi_logloss: 0.500226 + 0.00395742
[400]	cv_agg's multi_logloss: 0.444763 + 0.00408192
[500]	cv_agg's multi_logloss: 0.417129 + 0.00415751
499


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: type

## Just orign_raw data use(epoch 500)
[100]	cv_agg's multi_logloss: 0.951642 + 0.00310641
[200]	cv_agg's multi_logloss: 0.626394 + 0.00375563
[300]	cv_agg's multi_logloss: 0.500226 + 0.00395742
[400]	cv_agg's multi_logloss: 0.444763 + 0.00408192
[500]	cv_agg's multi_logloss: 0.417129 + 0.00415751
499
Wall time: 11min 5s

In [33]:
y_pred = model.predict(test_x)

In [34]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./data/dacon_g/submission_LGBM_3.csv', index=True)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras import optimizers


In [ ]:
%%time

# 1. 데이터셋 준비하기
x_train = scl.fit_transform(x_train)
y_train = to_categorical(y_train) # one-hot 인코딩
# y_train = y_train
# x_train = x_train.astype('float32')
# x_test = x_train[train_num:]
# y_test = y_train[train_num:]

# 2. 모델 구성하기
# model.reset_states()
model = Sequential()
model.add(Dense(256, kernel_initializer='random_uniform', input_shape=(3273,), activation='relu'))
model.add(Dense(256, kernel_initializer='random_uniform',  activation="relu"))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dense(198, kernel_initializer='random_uniform', activation='softmax'))

# 3. 모델 학습과정 설정하기
model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=["categorical_accuracy"])
 
# 4. 모델 학습시키기
hist = model.fit(x_train, y_train, epochs=1200, batch_size = 30, verbose=2)